# Load and prepare data

In [ ]:
import os
from pathlib import Path
import numpy as np

with open('X_train_original.npy', 'rb') as f:
    X_train_org = np.load(f)

with open('X_train_reverb_random.npy', 'rb') as f:
    X_train_reverb = np.load(f)
    
with open('X_test_reverb_random.npy', 'rb') as f:
    X_test_reverb = np.load(f)

In [ ]:
def normalize_data(data):
    data_min = np.min(data)
    data_max = np.max(data)
    data_norm = (data - data_min)/(data_max - data_min)
    return data_norm

In [ ]:
X_train_org_norm = normalize_data(X_train_org)
X_train_reverb_norm = normalize_data(X_train_reverb)
X_test_reverb_norm = normalize_data(X_test_reverb)

In [ ]:
import matplotlib.pyplot as plt

x = np.arange(0, 1., 1/16000)

plt.plot(x, X_train_org_norm[0])
plt.show()

print(X_train_reverb_norm.shape, X_train_org_norm.shape)

# Create Auto encoder model

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5*1024)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)
        
print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

fs = 16000
size = 8

def dice_coef(y_true, y_pred, smooth=2e-126):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true_f)) + K.sum(K.square(y_pred_f)) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

model_e2e = tf.keras.models.Sequential()
model_e2e.add(tf.keras.Input(shape=(fs,1))) #Make sure that the input size is the size of the signal
# model_e2e.add(tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True, )) #adjust input to unit variance and zero mean
#First Conv1D layer
model_e2e.add(tf.keras.layers.Conv1D(8, size, padding='same', activation='relu', strides=1)) #Convolve with 8 1D kernels of length size
model_e2e.add(tf.keras.layers.MaxPooling1D(2)) #Downsample - take the max out of every three elements
model_e2e.add(tf.keras.layers.Dropout(0.1)) 
model_e2e.add(tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True, )) #adjust input to unit variance and zero mean
#Second Conv1D layer
model_e2e.add(tf.keras.layers.Conv1D(16, size, padding='same', activation='relu', strides=1)) #Convolve with 16 1D kernels of length 11
model_e2e.add(tf.keras.layers.Conv1D(16, size, padding='same', activation='relu', strides=1)) #Convolve with 16 1D kernels of length 11
model_e2e.add(tf.keras.layers.MaxPooling1D(2))
model_e2e.add(tf.keras.layers.Dropout(0.2))
model_e2e.add(tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True, )) #adjust input to unit variance and zero mean
#Third Conv1D layer
model_e2e.add(tf.keras.layers.Conv1D(32, size, padding='same', activation='relu', strides=1))
model_e2e.add(tf.keras.layers.Conv1D(32, size, padding='same', activation='relu', strides=1))
model_e2e.add(tf.keras.layers.MaxPooling1D(2))
model_e2e.add(tf.keras.layers.Dropout(0.2))
model_e2e.add(tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True, )) #adjust input to unit variance and zero mean

model_e2e.add(tf.keras.layers.Conv1D(64, size, padding='same', activation='relu', strides=1))
model_e2e.add(tf.keras.layers.Conv1D(64, size, padding='same', activation='relu', strides=1))

model_e2e.add(tf.keras.layers.UpSampling1D(2))
model_e2e.add(tf.keras.layers.Conv1D(32, size, padding='same', activation='relu', strides=1))
model_e2e.add(tf.keras.layers.Conv1D(32, size, padding='same', activation='relu', strides=1))
model_e2e.add(tf.keras.layers.Dropout(0.2))
model_e2e.add(tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True, )) #adjust input to unit variance and zero mean

model_e2e.add(tf.keras.layers.UpSampling1D(2))
model_e2e.add(tf.keras.layers.Conv1D(16, size, padding='same', activation='relu', strides=1))
model_e2e.add(tf.keras.layers.Conv1D(16, size, padding='same', activation='relu', strides=1))
model_e2e.add(tf.keras.layers.Dropout(0.2))
model_e2e.add(tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True, )) #adjust input to unit variance and zero mean

model_e2e.add(tf.keras.layers.UpSampling1D(2))
model_e2e.add(tf.keras.layers.Conv1D(8, size, padding='same', activation='relu', strides=1))
model_e2e.add(tf.keras.layers.Conv1D(8, size, padding='same', activation='relu', strides=1))
model_e2e.add(tf.keras.layers.Dropout(0.1))
model_e2e.add(tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True, )) #adjust input to unit variance and zero mean

model_e2e.add(tf.keras.layers.Conv1D(1, 1, padding='same', strides=1))
model_e2e.add(tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=1e-3, center=True, scale=True, )) #adjust input to unit variance and zero mean

model_e2e.summary() #show breakdown of parameters

model_e2e.compile(loss=dice_coef_loss, optimizer='nadam', metrics=['mse']) #decide loss function and metrics


# Train model

In [ ]:
# Keep the first 100 sample for evaluation
hist = model_e2e.fit(
    x=X_train_reverb_norm[100:], 
    y=X_train_org_norm[100:],
    epochs=20, 
    batch_size=32)

In [ ]:
test_org_data = np.reshape( X_train_org_norm[25], (1, 16000,1))
test_reverb_data = np.reshape( X_train_reverb_norm[25], (1, 16000,1))
y_predict = model_e2e.predict(test_reverb_data)

# Do filter on 1 data

In [ ]:
import librosa
def denormalize_data(data, desired_min=-0.5, desired_max=0.5):
    flat_data = data.flatten()
    data_min = np.min(flat_data)
    data_max = np.max(flat_data)
    data_range = data_max - data_min
    desired_range = desired_max - desired_min
    
    x = []
    for i in range(len(flat_data)):
        entry = (flat_data[i] - data_min) / data_range * desired_range + desired_min
        x.append(entry)
        
    x = np.asarray(x)
    print(np.min(x), np.max(x))
    return x
    
y = denormalize_data(y_predict)

D = librosa.amplitude_to_db(np.abs(librosa.stft(y.flatten())), ref=np.max)

In [ ]:
import librosa

import librosa.display
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
librosa.display.specshow(D, y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('AutoEncoder')
plt.savefig('img/auto_encoder_fix.png')
plt.show()

In [ ]:
import scipy.io.wavfile as wav

y_int16 = denormalize_data(y, desired_min=-16384, desired_max=16383)

wav.write('./audio/autoencoder_fix.wav', 16000, y_int16.astype(np.int16))

# Do filter on all data

In [ ]:
train_data = []
for i in tnrange(len(X_train_reverb_norm)):
    denoised_signal = model_e2e.predict(X_train_reverb_norm[i])
    train_data.append(denoised_signal)
    
train_data = np.asarray(train_data)

In [ ]:
save_dir = Path('./E2E_data')
np.save(save_dir / "AE_train_reverb.npy", train_data)

In [ ]:
test_data = []
for i in tnrange(len(X_test_reverb_norm)):
    denoised_signal = model_e2e.predict(X_test_reverb_norm[i])
    test_data.append(denoised_signal)
    
test_data = np.asarray(test_data)

In [ ]:
save_dir = Path('./E2E_data')
np.save(save_dir / "AE_test_reverb.npy", test_data)